# Elektrostatisches Potential eines Kondensators

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Verwendung der Methode der finiten Differenzen zur Bestimmung des elektrostatischen Potentials eines Plattenkondensators in zwei Dimensionen demonstriert. Dazu ist die Laplace-Gleichung

$$
\left( \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} \right) \Phi(x,y) = 0
$$

mit den Randbedingungen

$$
\Phi(\boldsymbol{x}) = \begin{cases} 0 \qquad & \text{für $\boldsymbol{x}$ am Rand des betrachteten Gebiets} \\
+1 & \text{für $\boldsymbol{x}$ auf der oberen Platte des Kondensators} \\
-1 & \text{für $\boldsymbol{x}$ auf der unteren Platte des Kondensators}\end{cases} \,.
$$

zu lösen. Es werden sowohl die Jacobi- als auch die Gauß-Seidel-Methode implementiert.

## Importanweisungen

Zusätzlich zu den bereits bekannten Importanweisungen gibt es hier zwei neue Einträge.

Zunächst wird aus dem `collections`-Modul der Python-Standardbibliothek die Funktion `namedtuple` importiert. Dadurch wird es möglich, Einträge in einem Tupel auch mit einem Namen anzusprechen, um damit den Code verständlicher zu machen. Wir werden die geometrischen Informationen über den Kondensator in einem *named tuple* abspeichern.

Außerdem importieren wir aus dem SciPy-Paket das `ndimage`-Modul. Die Funktion `convolve` wird es uns erlauben, eine mehrdimensionale Faltung durchzuführen.

In [ ]:
from collections import namedtuple
import numpy as np
from scipy import ndimage
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Durchführung eines Schritts mit der Jacobi-Methode

Um die Mittelung über die vier benachbarten Gitterpunkte mit Hilfe einer Faltung durchführen zu können, werden die entsprechenden Gewichte im konstanten NumPy-Array `JAC_WEIGHTS` definiert. In jedem Jacobi-Schritt wird zunächst eine Mittelung auf dem ganzen Gitter durchgeführt, wodurch jedoch auch das Potential auf den Kondensatorplatten modifiziert wird. Daher müssen anschließend die Werte auf den Gitterplätzen, die zu den Kondensatorplatten gehören, wieder auf die durch die Randbedingungen gegebenen Werte gesetzt werden.

In [ ]:
JAC_WEIGHTS = 0.25*np.array([[0, 1, 0],
                             [1, 0, 1],
                             [0, 1, 0]])

def step_jacobi(v, cap):
    v = ndimage.convolve(v, JAC_WEIGHTS, mode="constant",
                         cval=0)
    v[cap.y_plus, cap.x_left:cap.x_right+1] = 1
    v[cap.y_minus, cap.x_left:cap.x_right+1] = -1
    return v

## Durchführung eines Schritts mit der Gauß-Seidel-Methode

Bei der Gauß-Seidel-Methode wird sukzessive das Potential an den einzelnen Gitterpunkten aktualisiert. Daher darf man in diesem Fall das Potential auf den Kondensatorplatten nicht verändern. Die Funktion `is_on_capacitor` dient dazu festzustellen, ob ein Gitterpunkt zu einer Kondensatorplatte gehört oder nicht. Bei der Durchführung eines Iterationsschritts in `step_gauss_seidel` wird zunächst das NumPy-Array `v` mit den anfänglichen Potentialwerten auf allen Seiten mit einer zusätzlichen Reihe von Gitterpunkten versehen, deren Potential gleich Null ist. Dadurch kann später die Mittelung in der gleichen Weise vorgenommen werden, unabhängig davon, ob der betreffende Gitterpunkt am Rand des ursprünglichen Gitters liegt oder nicht. Neben dem so erzeugten NumPy-Array `v_old` benötigt man noch ein zweites NumPy-Array `v_new`, in das die aktualisierten Werte geschrieben werden. Bei der Mittelung bedient man sich sowohl alter Werte, wenn der betreffende Gitterpunkt noch nicht aktualisiert wurde, als auch neuer Werte, wenn die Aktualisierung dort bereits erfolgt ist. Bei der Rückgabe des Resultats wird noch die eingangs hinzugefügte Berandung entfernt.

In [ ]:
def is_on_capacitor(nx, ny, cap):
    y_on_capacitor = ny in (cap.y_plus, cap.y_minus)
    x_on_capacitor = cap.x_left <= nx <= cap.x_right
    return y_on_capacitor and x_on_capacitor

In [ ]:
def step_gauss_seidel(v, cap):
    ny_max, nx_max = v.shape
    v_old = np.pad(v, 1)
    v_new = np.zeros((ny_max+1, nx_max+1))
    for ny in range(1, ny_max+1):
        for nx in range(1, nx_max+1):
            if is_on_capacitor(nx-1, ny-1, cap):
                v_new[ny, nx] = v_old[ny, nx]
            else:
                v_new[ny, nx] = (v_new[ny, nx-1]
                                 + v_new[ny-1, nx]
                                 + v_old[ny, nx+1]
                                 + v_old[ny+1, nx])/4
    return v_new[1:, 1:]

## Fehlerabschätzung

Um eine Abschätzung des Fehlers zu erhalten, wird das Integral

$$
\int \text{d}x \text{d}y \left[ \left( \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} \right) \Phi(x,y) \right]^2 \, 
$$

näherungsweise durch eine Summation über das Gitter ausgewertet. Dabei wird wieder eine Faltung verwendet, wobei nun die Gewichte in `RESIDUAL_WEIGHTS` hinterlegt sind. Vor der Auswertung der Summe über alle Gitterpunkte muss man den Fehler auf den Kondensatorplatten gleich null setzen, da dort das Potential den korrekten Wert annimmt ohne am Plattenrand der Laplace-Gleichung zu genügen. Neben dem Wert des Integrals wird auch das NumPy-Array `error` zurückgegeben, um später die räumliche Verteilung des Fehlers darstellen zu können.

In [ ]:
RESIDUAL_WEIGHTS = np.array([[0, -1, 0],
                             [-1, 4, -1],
                             [0, -1, 0]])

def residual(v, cap):
    error = ndimage.convolve(v, RESIDUAL_WEIGHTS,
                             mode="constant", cval=0)**2
    for y in (cap.y_plus, cap.y_minus):
        error[y, cap.x_left:cap.x_right+1] = 0
    error_sum = np.sum(error)
    return error, error_sum

## Durchführung der Iteration

Anfänglich wird das Potential $\Phi(x)$ überall auf null gesetzt, außer auf den beiden Kondensatorplatten, wo das Potential gemäß den Randbedingungen gleich $\pm 1$ ist. Anschließend werden Iterationsschritte mit der Funktion durchgeführt, die im Dictionary `STEP` zu dem Wert des Arguments `algorithm` gehört. Die Iterationsschleife wird bis zu einer maximalen Anzahl `n_iter_max` von Durchläufen abgearbeitet, sofern nicht zuvor die in `eps` übergebene Fehlerschranke durch den mit der Funktion `residual` erhaltenen Fehlerschätzer `error_sum` unterschritten wird. Diese Überprüfung erfolgt nur alle 100 Iterationsschritte, wobei auch der Fehlerschätzer zur Information ausgegeben wird.

In [ ]:
STEP = {"Jacobi": step_jacobi,
        "Gauß-Seidel": step_gauss_seidel
        }

def iterations(nx_max, ny_max, cap, n_iter_max, eps,
               algorithm):
    v = np.zeros((ny_max, nx_max))
    v[cap.y_plus, cap.x_left:cap.x_right+1] = 1
    v[cap.y_minus, cap.x_left:cap.x_right+1] = -1
    for n_iter in range(n_iter_max):
        v = STEP[algorithm](v, cap)
        if n_iter % 100 == 0:
            error, error_sum = residual(v, cap)
            print(n_iter, error_sum)
            if error_sum < eps:
                break
    return v, error

## Implementierung der Bedienelemente und graphische Darstellung des Potentials, des elektrischen Feldes oder des Fehlers

Mit den Bedienelementen können die folgenden Parameter eingestellt werden:
- `nx_max`: Ausdehnung des Gitters in $x$-Richtung
- `ny_max`: Ausdehnung des Gitters in $y$-Richtung
- `nx_cap`: Anzahl der Gitterpunkte in $x$-Richtung auf den beiden Kondensatorplatten
- `ny_cap`: Anzahl der Gitterpunkte in $y$-Richtung auf den beiden Kondensatorplatten
- `log_n_iter_max`: Zehnerlogarithmus der maximalen Zahl von Iterationsschritten
- `eps`: Fehlerschranke
- `algorithm`: zu verwendender Algorithmus, entweder Jacobi oder Gauß-Seidel

Die Funktion `plot_result` enthält noch eine innere Funktion, die es erlaubt, unterschiedliche Größen darzustellen, ohne das Potential jedesmal neu berechnen zu müssen. Dabei können die beiden folgenden Parameter
eingestellt werden:
- `fraction_display`: Anteil des darzustellenden Gitters, um in die Darstellung hineinzoomen zu können
- `quantity`: darzustellende Größe, entweder Potential, elektrisches Feld oder Fehler

In der Funktion `plot_inner` wird das elektrische Feld bei Bedarf aus dem bereits bestimmten Potential durch Bildung des Gradienten mit Hilfe der NumPy-Funktion `gradient` berechnet. Das Ergebnis wird durch Feldlinien dargestellt, die mit der matplotlib-Funktion `streamplot` bestimmt werden. Das Potential und der Fehler werden in zwei Dimensionen farblich dargestellt.

In [ ]:
wide_label = {"description_width": "initial"}
nitermax_label = r"$\log_{10}(n_\text{iter,max})$"

widget_dict = {"nx_max":
               widgets.IntSlider(
                   value=400, min=100, max=1000, step=50,
                   description=r"$n_{x,\text{max}}$"),
               "ny_max":
               widgets.IntSlider(
                   value=400, min=100, max=1000, step=50,
                   description=r"$n_{y,\text{max}}$"),
               "nx_cap":
               widgets.IntSlider(
                   value=50, min=5, max=100, step=1,
                   description=r"$n_{x,\text{capacitor}}$"),
               "ny_cap":
               widgets.IntSlider(
                   value=40, min=5, max=100, step=1,
                   description=r"$n_{y,\text{capacitor}}$"),
               "log_n_iter_max":
               widgets.IntSlider(
                   value=3, min=2, max=5, step=1,
                   description=nitermax_label,
                   style=wide_label),
               "eps":
               widgets.FloatLogSlider(
                   value=1e-6, min=-8, max=-1, step=1,
                   description=r"$\epsilon$"),
               "algorithm":
               widgets.RadioButtons(
                   options=["Jacobi", "Gauß-Seidel"],
                   value="Jacobi",
                   description="Algorithmus")
               }

widget_inner_dict = {"fraction_display":
                     widgets.FloatSlider(
                         value=1, min=0.1, max=1, step=0.1,
                         description=r"$f_\text{display}$"),
                     "quantity":
                     widgets.RadioButtons(
                         options=["Potential",
                                  "Elektrisches Feld",
                                  "Fehler"],
                         value="Potential",
                         description="Darstellung")
                     }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_result(nx_max, ny_max, nx_cap, ny_cap,
                log_n_iter_max, eps, algorithm):
    n_iter_max = 10**log_n_iter_max
    Capacitor = namedtuple("Capacitor",
                           ("y_plus", "y_minus",
                            "x_left", "x_right")
                           )
    cap = Capacitor((ny_max + ny_cap) // 2,
                    (ny_max - ny_cap) // 2,
                    (nx_max - nx_cap) // 2,
                    (nx_max + nx_cap) // 2)
    v, error_array = iterations(nx_max, ny_max, cap,
                                n_iter_max, eps, algorithm)

    @interact(**widget_inner_dict)
    def plot_inner(fraction_display, quantity):
        nx_plot = int((nx_max-1)*fraction_display/2)
        ny_plot = int((ny_max-1)*fraction_display/2)
        nx_half = nx_max // 2
        ny_half = ny_max // 2
        ny_bottom = ny_half - ny_plot
        ny_top = ny_half + ny_plot
        nx_left = nx_half - nx_plot
        nx_right = nx_half + nx_plot
        fig, ax = plt.subplots()
        if quantity == "Potential":
            v_plot = v[ny_bottom:ny_top+1,
                       nx_left:nx_right+1]
            potentialimg = ax.pcolormesh(v_plot)
            fig.colorbar(potentialimg, label="$V$")
        elif quantity == "Elektrisches Feld":
            xvals, yvals = np.meshgrid(
                np.arange(2*nx_plot+1),
                np.arange(2*ny_plot+1))
            efield_x, efield_y = np.gradient(-v)
            ax.streamplot(xvals, yvals,
                          efield_y[ny_bottom:ny_top+1,
                                   nx_left:nx_right+1],
                          efield_x[ny_bottom:ny_top+1,
                                   nx_left:nx_right+1],
                          density=2)
            for y in (cap.y_plus, cap.y_minus):
                ax.plot((cap.x_left-nx_left,
                         cap.x_right-nx_left),
                        (y-ny_bottom, y-ny_bottom),
                        color="black", lw=4)
        else:
            error_plot = error_array[ny_bottom:ny_top+1,
                                     nx_left:nx_right+1]
            errorimg = ax.pcolormesh(error_plot)
            fig.colorbar(errorimg, label="residual")
        ax.set_xlabel("$n_x$")
        ax.set_ylabel("$n_y$")
        ax.set_aspect("equal")